In [ ]:
import requests
import json
import psycopg2
import configparser
import sierra-ils-utils

In [ ]:
config = configparser.ConfigParser()
config.read('Y:\\SQL Reports\\creds\\api_info.ini')

'''
.ini file contains url/key/secret for the api in the following form
[api]
base_url = https://[local domain]/iii/sierra-api/v6
client_key = [enter Sierra API key]
client_secret = [enter Sierra API secret]
'''

base_url = config['api']['base_url'] + '/'
#note sierra-ils-utils assumes base_url contains the trailing /, which the file I have been using did not contain so it is appended here
client_key = config['api']['client_key']
client_secret = config['api']['client_secret']

sierra_api = sierra_ils_utils.SierraAPI(base_url,client_key,client_secret)
sierra_api.get('info/token')

In [ ]:
def runquery(query):
    config = configparser.ConfigParser()
    config.read('Y:\\SQL Reports\\creds\\api_info.ini')
    
    try:
	    conn = psycopg2.connect( config['api']['connection_string'] )
    except:
        print("unable to connect to the database")
        clear_connection()
        return
        
    cursor = conn.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    return rows

In [ ]:
def manual_charge(patron_id,amount,location):
    config = configparser.ConfigParser()
    config.read('C:\\SQL Reports\\creds\\api_info.ini')
    token = get_token()
    url = base_url + "/patrons/" + patron_id + "/fines/charge"
    params = {"amount": amount, "reason": "Residual fine","location": location}
    request = sierra_api.session.put(url, params)

def clear_fine(patron_id,invoiceNumber):
    config = configparser.ConfigParser()
    config.read('C:\\SQL Reports\\creds\\api_info.ini')
    token = get_token()
    url = base_url + "/patrons/" + patron_id + "/fines/payment"
    params = {"payments": [{"amount": 0, "paymentType": 2, "invoiceNumber": "" + invoiceNumber + ""}]}
    request = sierra_api.session.put(url, params)

In [ ]:
error_query = """\
    SELECT
      rm.record_num,
      (p.owed_amt * 100 - (SUM(COALESCE(f.item_charge_amt*100, 0) + COALESCE(f.processing_fee_amt*100, 0) + COALESCE(f.billing_fee_amt*100, 0) - COALESCE(f.paid_amt*100, 0))))::INT AS FineDiscrepancy,
      p.home_library_code AS location
    FROM sierra_view.record_metadata rm
    JOIN sierra_view.patron_record 
      ON p.id = rm.id
    LEFT JOIN sierra_view.fine f
      ON f.patron_record_id = p.id

    GROUP BY rm.record_num, p.owed_amt,3
    HAVING p.owed_amt != SUM(COALESCE(f.item_charge_amt, 0.00) + COALESCE(f.processing_fee_amt, 0.00) + COALESCE(f.billing_fee_amt, 0.00) - COALESCE(f.paid_amt, 0.00))
    """
    
manual_charge_query = """\
    SELECT
      rm.record_num,
      f.invoice_num::varchar
    FROM sierra_view.fine f
    JOIN sierra_view.record_metadata rm
        ON f.patron_record_id = rm.id
    WHERE f.assessed_gmt::DATE = CURRENT_DATE
        AND f.charge_code = '1'
        AND f.description = 'Residual fine' 
    """
    
#identify patrons with amt owed errors and create manual charges in the amount of those discrepancies
amt_owed_errors = runquery(error_query)
for rownum, row in enumerate(amt_owed_errors):
    manual_charge(str(row[0]),row[1],row[2])

#Find the newly created manual charges and waive them
fines_to_clear = runquery(manual_charge_query)
for rownum, row in enumerate(fines_to_clear):
    clear_fine(str(row[0]),row[1])         